In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import statistics
import math
import re

import warnings 
warnings.filterwarnings("ignore")

### Initial processing - Merge files and add the "Type" column

In [2]:
# # Add columns of each subfile
# df_url = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Web Scraping/urls/nha rieng, biet thu_url.csv")
# df_url['Loại hình'] = df_url['href'].apply(lambda x: 'Biệt thự' if 'biet-thu' in x else 'Nhà riêng')

# df_nharieng = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Web Scraping/data raw/data_nha rieng, biet thu.csv")
# df_nharieng_merged = pd.merge(df_nharieng, df_url['Loại hình'], left_index=True, right_index=True)
# df_nharieng_merged.to_csv('data_nha rieng, biet thu_loai hinh.csv', index=False)

# df_shophouse = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Web Scraping/data raw/data_shophouse.csv")
# df_shophouse['Loại hình'] = "Shophouse"
# df_shophouse.to_csv("data_shophouse_loai hinh.csv", index=False)

# df_matpho = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Web Scraping/data raw/data_matpho.csv")
# df_matpho['Loại hình'] = "Mặt phố"
# df_matpho.to_csv("Data_matpho_loai hinh.csv", index=False)

# # Merge into 1 file 
# merged_df = pd.concat([df_nharieng_merged, df_shophouse, df_matpho], ignore_index=True)
# merged_df.to_csv("data_loainhaban.csv", index=False)

### Data Cleaning

In [3]:
# Load data
dtset = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Dataset/data_loainhaban.csv")
dtset_columns = dtset.columns.tolist()
print("+ Shape: ", dtset.shape) # (Number of Lines, Number of Columns)
print("+ Column: ", dtset_columns) # Name the attribute columns.
dtset.info()

+ Shape:  (28781, 20)
+ Column:  ['Tiêu đề', 'Địa chỉ', 'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin', 'Người đăng', 'Thông tin mô tả', 'Diện tích', 'Mức giá', 'Mặt tiền', 'Đường vào', 'Hướng ban công', 'Hướng nhà', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý', 'Nội thất', 'Loại hình']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28781 entries, 0 to 28780
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Tiêu đề          28781 non-null  object
 1   Địa chỉ          28781 non-null  object
 2   Ngày đăng        28781 non-null  object
 3   Ngày hết hạn     28781 non-null  object
 4   Loại tin         28781 non-null  object
 5   Mã tin           28781 non-null  int64 
 6   Người đăng       28781 non-null  object
 7   Thông tin mô tả  28781 non-null  object
 8   Diện tích        28781 non-null  object
 9   Mức giá          28781 non-null  object
 10  Mặt tiền         18454 non-null  object
 11  Đường

In [4]:
dtset.head(3)

,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",25/02/2024,06/03/2024,Tin thường,39159568,Hoàng Thị Mai Thùy,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50 m²,"9,3 tỷ",NaN,NaN,NaN,NaN,5 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",29/02/2024,10/03/2024,Tin thường,38451107,Nguyễn Văn Tình,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264 m²,"33,5 tỷ",NaN,NaN,NaN,NaN,4 tầng,5 phòng,6 phòng,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự


In [5]:
dtset.tail(3)

,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
28778,"Bán nhà siêu VIP, mặt phố Tôn Đức Thắng, Đống ...","Phố Tôn Đức Thắng, Phường Quốc Tử Giám, Đốn...",01/03/2024,11/03/2024,Tin thường,39203647,Em Trường,"Mô tả:\n- Nhà mặt phố - lô góc, 2 mặt thoáng, ...",46 m²,"26,3 tỷ",NaN,NaN,NaN,NaN,4 tầng,3 phòng,4 phòng,Sổ đỏ,NaN,Mặt phố
28779,"Phố Đồng Cổ, chợ kinh doanh sầm uất cần bán","Đường Đồng Cổ, Phường Bưởi, Tây Hồ, Hà Nội",27/02/2024,08/03/2024,Tin thường,35029152,Nguyễn Quang Anh,"Phố Đồng Cổ, chợ kinh doanh sầm uất cần bán.\n...",51 m²,"10,5 tỷ","7,5 m",NaN,NaN,NaN,4 tầng,4 phòng,3 phòng,Sổ đỏ/ Sổ hồng,NaN,Mặt phố
28780,"Bán tòa nhà mặt phố Nguyễn Văn Cừ, Long Biên l...","Đường Nguyễn Văn Cừ, Phường Bồ Đề, Long Biên, ...",26/02/2024,07/03/2024,Tin thường,39072428,Nguyễn Thế Truyền,Tòa nhà mặt phố Nguyễn Văn Cừ - Long Biên mới ...,110 m²,65 tỷ,6 m,40 m,NaN,NaN,8 tầng,8 phòng,8 phòng,Sổ đỏ/ Sổ hồng,Đầy đủ.,Mặt phố


In [6]:
display(dtset[6:10])

,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
6,Đình Thôn - lô góc - ngõ ô tô - kinh doanh - 1...,"Đường Đình Thôn, Phường Mỹ Đình 1, Nam Từ Liêm...",26/02/2024,07/03/2024,Tin thường,39162771,Trường Dòng Tiền,Đình Thôn - lô góc - ngõ ô tô - kih doanh - 10...,90 m²,"28,6 tỷ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nhà riêng
7,Bán nhà 2.5 tầng diện tích 310m2 khu phố Từ Ho...,"Phố Từ Hoa, Phường Quảng An, Tây Hồ, Hà Nội",03/03/2024,13/03/2024,Tin thường,39134330,Pham Nhat Long,"+ Bán biệt thự, mặt Hồ Tây, mặt phố Từ Hoa, Qu...",310 m²,140 tỷ,7 m,3 m,Tây - Nam,Tây - Nam,2 tầng,4 phòng,4 phòng,Sổ đỏ/ Sổ hồng,NaN,Nhà riêng
8,"Nhà Đại La, dân xây, 41 m2, 4 tầng, gần mặt ph...","Phố Đại La, Phường Đồng Tâm, Hai Bà Trưng, Hà...",28/02/2024,09/03/2024,Tin thường,39186679,Phạm Công Vũ,"Cần bán ngay nhà Đại La, tầng 2 phòng ngủ, tho...",41 m²,"4,75 tỷ",5 m,3 m,NaN,NaN,4 tầng,4 phòng,3 phòng,Sổ đỏ,NaN,Nhà riêng
9,"Cần bán gấp nhà ngõ Liên Hoa 37m2 x 5 tầng, MT...","Phố Khâm Thiên, Phường Khâm Thiên, Đống Đa, Hà...",01/03/2024,11/03/2024,Tin thường,39204352,Đình Thắng,"Ngõ chùa Liên Hoa 37m² x 5 tầng, MT 5 mét, 3.6...",37 m²,"3,6 tỷ",5 m,3 m,NaN,NaN,5 tầng,3 phòng,4 phòng,Sổ Hồng chính Chủ,Cao cấp,Nhà riêng


In [7]:
# Display data by column
dtset.loc[:2, dtset_columns[0:2]]

,Tiêu đề,Địa chỉ
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ..."
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ..."
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo..."


In [8]:
print("Loại tin:", dtset[dtset_columns[4]].unique())
dtset.loc[:2, dtset_columns[2:7]]

Loại tin: ['Tin thường' 'Tin VIP Bạc' 'Tin VIP Vàng' 'Tin VIP Kim Cương']


,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng
0,25/02/2024,06/03/2024,Tin thường,39159568,Hoàng Thị Mai Thùy
1,29/02/2024,10/03/2024,Tin thường,38451107,Nguyễn Văn Tình
2,01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan


In [9]:
dtset.loc[:2, dtset_columns[7:12]]

,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào
0,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50 m²,"9,3 tỷ",NaN,NaN
1,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264 m²,"33,5 tỷ",NaN,NaN
2,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m


In [10]:
print("Hướng ban công:", dtset[dtset_columns[12]].unique())
print("Hướng nhà:", dtset[dtset_columns[13]].unique())

Hướng ban công: [nan 'Đông - Nam' 'Tây' 'Tây - Nam' 'Tây - Bắc' 'Đông - Bắc' 'Nam' 'Bắc'
 'Đông']
Hướng nhà: [nan 'Đông - Nam' 'Tây' 'Tây - Nam' 'Đông - Bắc' 'Tây - Bắc' 'Nam' 'Bắc'
 'Đông']


In [11]:
dtset.loc[:2, dtset_columns[14:20]]

,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,5 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,4 tầng,5 phòng,6 phòng,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự


In [12]:
print("Pháp lý:", dtset[dtset_columns[17]].unique())

Pháp lý: ['Sổ đỏ/ Sổ hồng' nan 'Sổ đỏ/ Sổ hồng.' 'Sổ đỏ' 'Sổ Hồng chính Chủ'
 'Hợp đồng mua bán' 'Đã có sổ đỏ' 'Sổ đỏ phân lô chính chủ.' 'Sổ đ'
 'Sổ đỏ vuông.' 'Sổ đỏ,sổ hồng' 'Sổ đỏ.' 'Sổ đỏ pháp lý , miễm trung gian'
 'Sổ đỏ cất két, sẵn sàng giao dịch' 'Đã có sổ.' 'Chính chủ.'
 'Sổ đỏ lô góc biệt thự không lỗi phong thuỷ' 'Đã có sổ' 'Sổ đỏ chính chủ'
 'Sổ đỏ cất két sang tên luôn' 'Sổ đỏ vuông vắn' 'Sổ hồng' 'Đang chờ sổ'
 'Sổ đỏ vuông đẹp giao dịch ngay' 'Sđcc sang tên luôn' 'Sổ đỏ rõ ràng'
 'Sổ chính chủ' 'Sổ Hồng riêng' '- Sổ đỏ chính chủ!' 'Có sổ.'
 'Rổ riêng cất két' 'Sđcc phân lô' 'Sổ đỏ chính chủ.' 'Sổ đỏ, sổ hồng.'
 'Sdcc' 'Đã có sổ đỏ.' 'Đã có sổ đỏ chính chủ.' 'Sổ đỏ lâu dài.'
 'Sổ chính chủ vuông đẹp' 'Sổ Đỏ' 'Sổ đỏ chính chủ sẵn giao dịch'
 'Sổ đỏ vuông đẹp' 'Đầy đủ' 'Sổ đỏ lâu dài' 'Sổ chính chủ giao dịch ngay'
 'Sổ đỏ đẹp' 'Sổ đỏ chinh chủ' 'Sổ đỏ sẵn sàng sang tên.'
 'Sổ đỏ chính chủ, pháp lý sạch' 'SỔ đỏ chính chủ' 'Hợp đồng mua bán.'
 'HĐMB' 'SĐCC' 'Sổ Đỏ/Sổ Hồng C

### Dealing with Duplicate lines

In [13]:
# Check home listing templates with the same "Mã tin":
matin_duplicate_df = dtset[dtset["Mã tin"].duplicated()]
print("Number of lines duplicated 'Mã tin' =", len(matin_duplicate_df))
matin_duplicate_df[31:33]

Number of lines duplicated 'Mã tin' = 123


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
4566,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
4814,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng


In [14]:
# Delete duplicates
# Check if any data is duplicated
duplicate_df = dtset[dtset.duplicated()] # Save duplicate lines to separate dataframe
duplicate_df = duplicate_df.reset_index() # Reset the index of the new dataframe.
#display(duplicate_df)
print("Number of duplicate lines =", len(duplicate_df))

Number of duplicate lines = 108


In [15]:
# Example of all lines with duplicate "Địa chỉ" of a data sample
diachivalue = duplicate_df["Địa chỉ"].iloc[0] # Get the value of the "Địa chỉ" column from the row at index 0 in the dataframe containing duplicates
print("Data Type =", type(diachivalue)) # This value has a String data type
dtset.loc[dtset['Địa chỉ'] == diachivalue] # Return to the original dataset, display all rows that are duplicates

Data Type = <class 'str'>


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
16,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
109,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
115,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
170,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21532,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21602,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21650,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng


In [16]:
# Example of all duplicate lines "Tiêu đề" of a data sample
tieudevalue = duplicate_df["Tiêu đề"].iloc[0] # Get the value of the "Tiêu đề" column from the row at index 0 in the dataframe containing duplicates
print("Data Type =", type(tieudevalue)) # This value has a String data type
dtset.loc[dtset["Tiêu đề"] == tieudevalue] # Return to the original dataset, display all rows that are duplicates

Data Type = <class 'str'>


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
16,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
109,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
115,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
170,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21532,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21602,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
21650,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng


"Địa chỉ" of some stories may coincide, but "Tiêu đề" and, most importantly, "Mức giá" are not. It's normal for this template to have duplicate addresses, especially when we've dealt with the original raw data by removing some attribute columns (like Tiêu đề, Người đăng, Loại tin, Mã tin) and adjusting the "Số phòng ngủ", "Số tầng", "Số toliet", etc. so that they are not NaN, empty or Missing.

In [17]:
# Except for the original row, remove all subsequent rows that are duplicates:
print("Initial dataset size:", dtset.shape)
dtset = dtset.drop_duplicates()
dtset = dtset.reset_index(drop=True)
print("Size after removing duplicate rows:", dtset.shape)

Initial dataset size: (28781, 20)
Size after removing duplicate rows: (28673, 20)


There exist lines with the same 'Mã tin' but different 'Ngày đăng' and 'Ngày hết hạn', making them separate home sale listings. Therefore, we must check the duplicate again and delete the messages with the same 'Mã tin'.

In [18]:
# Check for duplicate again and delete messages with the same "Mã tin"
# Check the property listings with the same "Mã tin"
matin_duplicate_df = dtset[dtset["Mã tin"].duplicated()]
print("Rows with similar Mã tin:", len(matin_duplicate_df))
matin_duplicate_df.head(3)

Rows with similar Mã tin: 15


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
108,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
9705,Chuyên sâu biệt thự từ 135-200-300m2 (giá tốt)...,"Dự án An Lạc Green Symphony, Xã Vân Canh, Hoài...",26/02/2024,12/03/2024,Tin VIP Kim Cương,38896057,Đỗ Thanh Thúy,Đầu xuân Giáp Thìn 2024 chúc quý anh chị nhiều...,210 m²,125 triệu/m²,NaN,NaN,NaN,Đông - Nam,NaN,NaN,NaN,NaN,NaN,Biệt thự
12866,"Bán nhà 32m2 x 5 tầng, sổ đỏ, tại phố La Nội, ...","Đường La Nội, Phường Dương Nội, Hà Đông, Hà Nội",01/03/2024,08/03/2024,Tin thường,39140138,Anh Tuấn,"Mình cần bán nhà tại phố La Nội, phường Dương ...",32 m²,"3,6 tỷ",NaN,NaN,NaN,NaN,5 tầng,3 phòng,4 phòng,Sổ đỏ,Đầy đủ,Nhà riêng


In [19]:
# Example of all rows with duplicate "Mã tin" for a sample data
matinvalue = matin_duplicate_df["Mã tin"].iloc[0] # Get the value of "Mã tin" from the row at index 0 in the dataframe containing duplicates
print("Data Type =", type(matinvalue)) # This value has an integer data type
dtset.loc[dtset["Mã tin"] == matinvalue] # Return to the original dataset, display all rows with the same "Mã tin"

Data Type = <class 'numpy.int64'>


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự
108,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
14052,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",14/03/2024,24/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Nhà riêng
14868,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",14/03/2024,24/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự


In [20]:
MaTin_Index_ToDelete = [] # Create a list variable to store the indices of these duplicates
for ind in matin_duplicate_df.index:
  MaTin_Index_ToDelete.append(ind)
print("Index Count =", len(MaTin_Index_ToDelete))
print("Index List:", MaTin_Index_ToDelete) # From this list variable, we can return to the original dataset to delete
# rows starting from the second occurrence that have the same "Mã tin" as the first occurrence

Index Count = 15
Index List: [108, 9705, 12866, 14052, 14868, 20048, 20051, 20053, 20063, 20064, 23615, 25804, 26385, 27226, 27363]


In [21]:
print("Initial dataset size:", dtset.shape)
for ind in MaTin_Index_ToDelete:
  dtset = dtset.drop(ind)
dtset = dtset.reset_index(drop=True)
print("Size after removing duplicate rows:", dtset.shape)

Initial dataset size: (28673, 20)


Size after removing duplicate rows: (28658, 20)


###  Attribute columns

Missing Data

In [22]:
print("Dataset shape:", dtset.shape)

Dataset shape: (28658, 20)


In [23]:
# Check the number of missing values in each column as a percentage:
missing_percentage = dtset.isnull().sum() / dtset.shape[0] * 100.00
missing_percentage

Tiêu đề             0.000000
Địa chỉ             0.000000
Ngày đăng           0.000000
Ngày hết hạn        0.000000
Loại tin            0.000000
Mã tin              0.000000
Người đăng          0.000000
Thông tin mô tả     0.000000
Diện tích           0.000000
Mức giá             0.000000
Mặt tiền           35.975993
Đường vào          52.372810
Hướng ban công     87.455510
Hướng nhà          79.604299
Số tầng            17.213344
Số phòng ngủ       33.236793
Số toilet          39.981855
Pháp lý            19.530323
Nội thất           45.585875
Loại hình           0.000000
dtype: float64

In [24]:
# Check unique column "Pháp lý"
phaply = dtset["Pháp lý"].copy()
phaply_UniqueCount = phaply.value_counts()
print(phaply_UniqueCount)

Pháp lý
Sổ đỏ/ Sổ hồng                                        15797
Sổ đỏ/ Sổ hồng.                                        1917
Sổ đỏ                                                  1818
Sổ đỏ.                                                  610
Hợp đồng mua bán                                        419
                                                      ...  
Đã có sổ đỏ , tặng full nội thất nhập ngoại               1
sổ đỏ                                                     1
Sổ đỏ chính chủ , vuông đẹp nở hậu, giao dịch ngay        1
Sổ đỏ chính chủ sẵn sàng giao dịch .                      1
Sổ đỏ chính chủ . Chuẩn chỉ cực đẹp                       1
Name: count, Length: 564, dtype: int64


As for "Pháp lý", this is information that the seller or broker selling the house freely declares, has no specific rules, presents content that is not related to papers or even does not declare anything. Because the values of "Legal" are quite diverse with nearly 600 unique values and complex data processing with  "Vi bằng", "Hợp đồng mua bán", "Hợp đồng đặt cọc", "Giấy tay", "Sổ hồng" hay "Sổ đỏ" all have separate legal meanings. 

### Format number

Create a new DataFrame 'df' as a copy of the original DataFrame 'dtset'

In [25]:
# Create a new DataFrame 'df' as a copy of the original DataFrame 'dtset'
df = dtset.loc[:, :].copy()

In [26]:
# Function to convert "Ngày đăng" and "Ngày hết hạn"
def convert_dates(df):
    df['Ngày đăng'] = pd.to_datetime(df['Ngày đăng'], format='%d/%m/%Y')
    df['Ngày hết hạn'] = pd.to_datetime(df['Ngày hết hạn'], format='%d/%m/%Y')
    return df

# Assuming df is your DataFrame
df = convert_dates(df)

In [27]:
# Function to extract area from 'Diện tích' column and convert it to float
def get_area(dientich):
    if isinstance(dientich, float) or isinstance(dientich, int):  # Check if the value is already a number
        return dientich  # Return the original value if it's already a number
    else:
        # Remove commas from the string
        area = dientich.replace(',', '')
        space_index = area.find(' ')
        area = area[:space_index].strip()  # Extract numerical part
        return float(area)  # Convert to float

# Function to format prices in 'Mức giá' column and return as float
def format_price(x):
    result = x['Mức giá']  # Extract price value
    area = x['Diện tích']  # Extract area value for per square meter price calculation
    
    result = result.replace(',', '.')  # Replace comma with dot for decimal handling
    
    # Check if the result is NaN
    if pd.isnull(result):
        return np.nan
    
    # Check if the result contains '/' indicating a unit
    if '/' in result:
        return result  # Return the original value
    
    # Check if the result contains 'triệu/m²'
    if 'triệu/m²' in result:
        # Convert to float and round to one decimal place for 'triệu/m²' prices
        return round(float(result.replace(' triệu/m²', '')), 1)
    
    # Check if the result contains 'triệu'
    if 'triệu' in result:
        # Convert to float and adjust for area if necessary, rounding to one decimal place
        return round(float(result.replace(' triệu', '')) / area, 1)
    
    # Check if the result contains 'tỷ'
    if 'tỷ' in result:
        # Convert to float, adjust for area if necessary, and convert to million, rounding to one decimal place
        return round(float(result.replace(' tỷ', '')) * 1000 / area, 1)
    
    # Try converting to float, handle errors, and round to one decimal place
    try:
        return round(float(result), 1)
    except ValueError:
        # If blank, 'Thỏa thuận', or NaN
        return result  

# Function to convert 'Số phòng ngủ' and 'Số toilet' values to int
def convert_to_int(x):
    if isinstance(x, str) and 'phòng' in x:
        return int(x.replace(' phòng', ''))
    elif isinstance(x, str) and 'tầng' in x:
        return int(x.replace(' tầng', ''))
    else:
        return np.nan

# Applying get_area function to the 'Diện tích' column
df['Diện tích'] = df['Diện tích'].apply(get_area)

# Applying format_price function to the 'Mức giá' column
df['Mức giá'] = df.apply(format_price, axis=1)

# Applying convert_to_int function to the 'Số phòng ngủ' and 'Số toilet' and 'Số tầng' columns
df['Số phòng ngủ'] = df['Số phòng ngủ'].apply(convert_to_int)
df['Số toilet'] = df['Số toilet'].apply(convert_to_int)
df['Số tầng'] = df['Số tầng'].apply(convert_to_int)

df.head(3)

,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",2024-02-25,2024-03-06,Tin thường,39159568,Hoàng Thị Mai Thùy,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",2024-02-29,2024-03-10,Tin thường,38451107,Nguyễn Văn Tình,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",2024-03-01,2024-03-11,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28658 entries, 0 to 28657
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Tiêu đề          28658 non-null  object        
 1   Địa chỉ          28658 non-null  object        
 2   Ngày đăng        28658 non-null  datetime64[ns]
 3   Ngày hết hạn     28658 non-null  datetime64[ns]
 4   Loại tin         28658 non-null  object        
 5   Mã tin           28658 non-null  int64         
 6   Người đăng       28658 non-null  object        
 7   Thông tin mô tả  28658 non-null  object        
 8   Diện tích        28658 non-null  float64       
 9   Mức giá          28658 non-null  object        
 10  Mặt tiền         18348 non-null  object        
 11  Đường vào        13649 non-null  object        
 12  Hướng ban công   3595 non-null   object        
 13  Hướng nhà        5845 non-null   object        
 14  Số tầng          23725 non-null  float

In [29]:
# The original dataset
print(dtset.shape)
dtset.head(3)  

(28658, 20)


,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",25/02/2024,06/03/2024,Tin thường,39159568,Hoàng Thị Mai Thùy,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50 m²,"9,3 tỷ",NaN,NaN,NaN,NaN,5 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",29/02/2024,10/03/2024,Tin thường,38451107,Nguyễn Văn Tình,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264 m²,"33,5 tỷ",NaN,NaN,NaN,NaN,4 tầng,5 phòng,6 phòng,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",01/03/2024,11/03/2024,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99 m²,12 tỷ,5 m,15 m,Đông - Nam,Đông - Nam,4 tầng,4 phòng,4 phòng,NaN,NaN,Biệt thự


In [30]:
df

,Tiêu đề,Địa chỉ,Ngày đăng,Ngày hết hạn,Loại tin,Mã tin,Người đăng,Thông tin mô tả,Diện tích,Mức giá,Mặt tiền,Đường vào,Hướng ban công,Hướng nhà,Số tầng,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình
0,"Chính chủ bán nhà Dương Nội, Hà Đông, phân lô,...","Dự án KĐTM Dương Nội, Đường Tố Hữu, Phường Dươ...",2024-02-25,2024-03-06,Tin thường,39159568,Hoàng Thị Mai Thùy,Cần bán gấp nhà Aeon Dương Nội Hà Đông nhà ở l...,50.0,186.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
1,Bán gấp siêu biệt thự khu A - An Khang Villa D...,"Dự án An Khang Villa, Đường Tố Hữu, Phường La ...",2024-02-29,2024-03-10,Tin thường,38451107,Nguyễn Văn Tình,"Hạ sốc 1.5 tỷ, giá chào mới 33.5 tỷ. (giá chào...",264.0,126.9,NaN,NaN,NaN,NaN,4.0,5.0,6.0,Sổ đỏ/ Sổ hồng,NaN,Biệt thự
2,"Quỹ hàng biệt thự - liền kề vị trí đẹp, giá tố...","Dự án Hado Charm Villas, Đường Đại lộ Thăng Lo...",2024-03-01,2024-03-11,Tin thường,39204669,Chinh Phan,"Em bán quỹ biệt thự - liền kề đẹp, giá tốt, mu...",99.0,121.2,5 m,15 m,Đông - Nam,Đông - Nam,4.0,4.0,4.0,NaN,NaN,Biệt thự
3,"Bán nhà 30m2 x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...","Phố Cầu Cốc, Phường Tây Mỗ, Nam Từ Liêm, Hà Nội.",2024-03-02,2024-03-12,Tin thường,39214622,Phạm Dương,"Bán nhà 30m² x 5 tầng Cầu Cốc, Tây Mỗ, cách cổ...",30.0,113.3,"3,54 m",2 m,Tây,Tây,5.0,3.0,4.0,Sổ đỏ/ Sổ hồng.,Đầy đủ,Nhà riêng
4,"Bán biệt thự liền kề Ba Đình, nội thất dát vàn...","Dự án 671 Hoàng Hoa Thám, Đường Hoàng Hoa Thám...",2024-02-25,2024-03-06,Tin thường,38856881,Trịnh Văn Điệp,Bán căn biệt thự liền kề Viglacera Ba Đình. Hà...,90.0,305.6,"5,2 m",10 m,NaN,NaN,5.0,6.0,6.0,Sổ đỏ/ Sổ hồng,Đầy đủ,Biệt thự
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28653,"(Ảnh thật) bán gấp nhà mặt phố Tây Sơn, 5 tầng...","Đường Tây Sơn, Phường Yết Kiêu, Hà Đông, Hà Nội",2024-03-04,2024-03-14,Tin thường,39229326,Lê Thị Ngọc Bích,"- Vị trí trung tâm quận Hà Đông, khu vực đang ...",409.0,21.7,5 m,NaN,NaN,NaN,5.0,3.0,4.0,Sổ đỏ,Đầy đủ,Mặt phố
28654,"Bán nhà mặt phố Hoàng Cầu, Thái Hà 90 tỷ 220m2...","Phố Hoàng Cầu, Phường Ô Chợ Dừa, Đống Đa, Hà...",2024-03-03,2024-03-13,Tin thường,38672743,Anh Tuấn,"Bán tòa nhà mặt phố khu Thái Hà, Hoàng Cầu diệ...",220.0,409.1,"8,5 m",NaN,NaN,NaN,8.0,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,Mặt phố
28655,"Bán nhà siêu VIP, mặt phố Tôn Đức Thắng, Đống ...","Phố Tôn Đức Thắng, Phường Quốc Tử Giám, Đốn...",2024-03-01,2024-03-11,Tin thường,39203647,Em Trường,"Mô tả:\n- Nhà mặt phố - lô góc, 2 mặt thoáng, ...",46.0,571.7,NaN,NaN,NaN,NaN,4.0,3.0,4.0,Sổ đỏ,NaN,Mặt phố
28656,"Phố Đồng Cổ, chợ kinh doanh sầm uất cần bán","Đường Đồng Cổ, Phường Bưởi, Tây Hồ, Hà Nội",2024-02-27,2024-03-08,Tin thường,35029152,Nguyễn Quang Anh,"Phố Đồng Cổ, chợ kinh doanh sầm uất cần bán.\n...",51.0,205.9,"7,5 m",NaN,NaN,NaN,4.0,4.0,3.0,Sổ đỏ/ Sổ hồng,NaN,Mặt phố


In [32]:
# Save the DataFrame to a CSV file 
df.to_csv('nhaban_clean.csv', index=False)